<a href="https://colab.research.google.com/github/TiagoNLopez/stuff/blob/master/GO_sim_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resúmen del trabajo sobre medidas de similaridad semántica

Al trabajar con la ontología genética es necesario una medida que permita comparar términos de la misma, de esta necesidad surgen las medidas de similaridad semántica: funciones que dados dos elementos de la ontología (o dos conjuntos de elementos) devuelven un valor que indica la cercanía dentro del grafo. Dentro de las medidas más relevantes se encuentra las de Resnik, Lin y Schlicker (para comparar funciones) y el promedio, el mínimo, el máximo de las anteriores, junto con la medida de coseno (para comparar conjuntos de funciones). Todas estas medidas tienen aciertos y errores con respecto a que información de la ontología logran representar, esto se puede identificar cuando se realizan pruebas, puesto que tampoco se cuenta con una medida estándar de evaluación de dichas medidas. Por esto la importancia de encontrar una medida de similaridad que contenga toda la información de la ontología y que, de ser posible, tenga una forma directa de medir su bondad.


# Análisis de similaridades semánticas

El siguiente código carga la imagen del fragmento de la GO a analizar:
![alt text](https://raw.githubusercontent.com/TiagoNLopez/stuff/master/parte%20de%20ontologia%20con%20etiquetas.png)



# Similaridad sobre funciones individuales

Para este análisis propongo los genes:

**G1:  GO:0012501**,  **GO:0006915**    

**G2:  GO:0048519**,  **GO:0048523**

**G3:  GO:0050794**,  **GO:0010941**

Estos genes comparten funciones en los mismos niveles, que si bien en el gráfico parecen distintos más adelante se demuestra en código que son los mismos.



---


## Medidas de similaridad entre funciones

Las medidas que voy a utilizar son las siguientes:

**Resnik**
$$ Sim_{Res}(c_1,c_2)=IC(c_{MICA}) $$

**Lin**
$$ Sim_{Lin}(c_1,c_2)=\frac{2\times IC(c_{MICA})}{IC(c_1)+IC(c_2)} $$

**Jiang, Conrath**
$$ Sim_{JC}(c_1,c_2)=1-IC(c_1)+IC(c_2)-2\times IC(c_{MICA}) $$

**Schlicker**
$$ Sim_{Rel}(c_1,c_2)=Sim_{Lin}(c_1,c_2) \times (1-p(c_{MICA})) $$



---


## Mediciones a realizar

Se van a comparar las funciones de **G2** con respecto a **G1** y **G2**, siempre en un mismo nivel. Los resultados se obtienen en el código siguiente.

In [0]:
# instalar goatools y descargar go
!pip install goatools
!wget http://purl.obolibrary.org/obo/go/go-basic.obo

--2019-10-16 16:55:43--  http://purl.obolibrary.org/obo/go/go-basic.obo
Resolving purl.obolibrary.org (purl.obolibrary.org)... 52.3.123.63
Connecting to purl.obolibrary.org (purl.obolibrary.org)|52.3.123.63|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://current.geneontology.org/ontology/go-basic.obo [following]
--2019-10-16 16:55:43--  http://current.geneontology.org/ontology/go-basic.obo
Resolving current.geneontology.org (current.geneontology.org)... 13.35.254.69, 13.35.254.158, 13.35.254.85, ...
Connecting to current.geneontology.org (current.geneontology.org)|13.35.254.69|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31324352 (30M) [text/obo]
Saving to: ‘go-basic.obo.1’

go-basic.obo.1      100%[===================>]  29.87M  99.9MB/s    in 0.3s    

2019-10-16 16:55:44 (99.9 MB/s) - ‘go-basic.obo.1’ saved [31324352/31324352]



In [0]:
from goatools.obo_parser import GODag
godag=GODag("go-basic.obo")

go-basic.obo: fmt(1.2) rel(2019-10-07) 47,285 GO Terms


In [0]:
# comprobar anotaciones y niveles

G1F1_go_id = 'GO:0012501'
G1F2_go_id = 'GO:0006915'

G2F1_go_id = 'GO:0048519'
G2F2_go_id = 'GO:0048523'

G3F1_go_id = 'GO:0050794'
G3F2_go_id = 'GO:0010941'

print('Gen 1')
print(godag[G1F1_go_id])
print(godag[G1F2_go_id])

print('Gen 2')
print(godag[G2F1_go_id])
print(godag[G2F2_go_id])

print('Gen 3')
print(godag[G3F1_go_id])
print(godag[G3F2_go_id])

Gen 1
GO:0012501	level-03	depth-03	programmed cell death [biological_process]
GO:0006915	level-04	depth-04	apoptotic process [biological_process]
Gen 2
GO:0048519	level-03	depth-03	negative regulation of biological process [biological_process]
GO:0048523	level-04	depth-04	negative regulation of cellular process [biological_process]
Gen 3
GO:0050794	level-03	depth-03	regulation of cellular process [biological_process]
GO:0010941	level-04	depth-04	regulation of cell death [biological_process]


In [0]:
import os
from goatools.associations import dnld_assc
fin_gaf = os.path.join(os.getcwd(), "tair.gaf")
associations = dnld_assc(fin_gaf, godag)

HMS:0:00:08.172170 233,420 annotations READ: /content/tair.gaf 
19363 IDs in loaded association branch, BP


In [0]:
# ancestro común más informativo
from goatools.semantic import deepest_common_ancestor
go_root = deepest_common_ancestor([G2F1_go_id, G3F1_go_id], godag)
print(go_root)

GO:0050789


In [0]:
# probar y comparar el contenido de información
from goatools.semantic import TermCounts, get_info_content
termcounts = TermCounts(godag, associations)
infocontent1 = get_info_content(G1F1_go_id, termcounts)
infocontent2 = get_info_content(G1F2_go_id, termcounts)
print('Contenido de información padre ({}) = {}'.format(G1F1_go_id, infocontent1))
print('Contenido de información hijo ({}) = {}'.format(G1F2_go_id, infocontent2))

Contenido de información padre (GO:0012501) = 2.2416496666707166
Contenido de información hijo (GO:0006915) = 3.985942649793393


In [0]:
# similaridad JC

def JC_sim(g1, g2, go, terms):
  ic_g1 = get_info_content(g1, terms)
  ic_g2 = get_info_content(g2, terms)
  sim_res = resnik_sim(g1, g2, go, terms)
  sim_JC = 1 - ic_g1 + ic_g2 - 2 * sim_res
  return(sim_JC)

# similaridad rel
import math

def rel_sim(g1, g2, go, terms):
  sim_lin = lin_sim(g1, g2, go, terms)
  sim_res = resnik_sim(g1, g2, go, terms)
  # obtener p(c_MICA)
  p_mica = math.exp((-sim_res))
  sim_rel = sim_lin * (1-p_mica)
  return(sim_rel)

In [0]:
from goatools.semantic import TermCounts, get_info_content, resnik_sim, lin_sim
termcounts = TermCounts(godag, associations)

# similaridades entre términos de nivel 3
res_G1F1_vs_G2F1 = resnik_sim(G1F1_go_id, G2F1_go_id, godag, termcounts)
res_G2F1_vs_G3F1 = resnik_sim(G2F1_go_id, G3F1_go_id, godag, termcounts)

lin_G1F1_vs_G2F1 = lin_sim(G1F1_go_id, G2F1_go_id, godag, termcounts)
lin_G2F1_vs_G3F1 = lin_sim(G2F1_go_id, G3F1_go_id, godag, termcounts)

jc_G1F1_vs_G2F1 = JC_sim(G1F1_go_id, G2F1_go_id, godag, termcounts)
jc_G2F1_vs_G3F1 = JC_sim(G2F1_go_id, G3F1_go_id, godag, termcounts)

rel_G1F1_vs_G2F1 = rel_sim(G1F1_go_id, G2F1_go_id, godag, termcounts)
rel_G2F1_vs_G3F1 = rel_sim(G2F1_go_id, G3F1_go_id, godag, termcounts)

# similaridades entre términos de nivel 4
res_G1F2_vs_G2F2 = resnik_sim(G1F2_go_id, G2F2_go_id, godag, termcounts)
res_G2F2_vs_G3F2 = resnik_sim(G2F2_go_id, G3F2_go_id, godag, termcounts)

lin_G1F2_vs_G2F2 = lin_sim(G1F2_go_id, G2F2_go_id, godag, termcounts)
lin_G2F2_vs_G3F2 = lin_sim(G2F2_go_id, G3F2_go_id, godag, termcounts)

jc_G1F2_vs_G2F2 = JC_sim(G1F2_go_id, G2F2_go_id, godag, termcounts)
jc_G2F2_vs_G3F2 = JC_sim(G2F2_go_id, G3F2_go_id, godag, termcounts)

rel_G1F2_vs_G2F2 = rel_sim(G1F2_go_id, G2F2_go_id, godag, termcounts)
rel_G2F2_vs_G3F2 = rel_sim(G2F2_go_id, G3F2_go_id, godag, termcounts)

print("Comparaciones contra G2F1: GO:0048519 (lvl 3)")
print("Measure   vs G1F1 GO:0012501  vs G3F1 GO:0050794")
print("--------- ------------------- -------------------")
print("resnik    {sim1:.16f} {sim2:.16f}".format(sim1=res_G1F1_vs_G2F1, sim2=res_G2F1_vs_G3F1))
print("lin       {sim1:.16f} {sim2:.16f}".format(sim1=lin_G1F1_vs_G2F1, sim2=lin_G2F1_vs_G3F1))
print("JC        {sim1:.16f} {sim2:.16f}".format(sim1=jc_G1F1_vs_G2F1, sim2=jc_G2F1_vs_G3F1))
print("relevance {sim1:.16f} {sim2:.16f}".format(sim1=rel_G1F1_vs_G2F1, sim2=rel_G2F1_vs_G3F1))
print(" ")
print("Comparaciones contra G2F2: GO:0048523 (lvl 4)")
print("Measure   vs G1F2 GO:0006915  vs G3F2 GO:0010941")
print("--------- ------------------- -------------------")
print("resnik    {sim1:.16f} {sim2:.16f}".format(sim1=res_G1F2_vs_G2F2, sim2=res_G2F2_vs_G3F2))
print("lin       {sim1:.16f} {sim2:.16f}".format(sim1=lin_G1F2_vs_G2F2, sim2=lin_G2F2_vs_G3F2))
print("JC        {sim1:.16f} {sim2:.16f}".format(sim1=jc_G1F2_vs_G2F2, sim2=jc_G2F2_vs_G3F2))
print("relevance {sim1:.16f} {sim2:.16f}".format(sim1=rel_G1F2_vs_G2F2, sim2=rel_G2F2_vs_G3F2))

Comparaciones contra G2F1: GO:0048519 (lvl 3)
Measure   vs G1F1 GO:0012501  vs G3F1 GO:0050794
--------- ------------------- -------------------
resnik    -0.0000000000000000 0.5447189755507806
lin       -0.0000000000000000 0.5922144342743970
JC        -0.0119626656315570 -0.7092114840755080
relevance -0.0000000000000000 0.2487272458493268
 
Comparaciones contra G2F2: GO:0048523 (lvl 4)
Measure   vs G1F2 GO:0006915  vs G3F2 GO:0010941
--------- ------------------- -------------------
resnik    -0.0000000000000000 0.6099134680652128
lin       -0.0000000000000000 0.3221751633240093
JC        -1.5676144427308445 0.7297394098883263
relevance -0.0000000000000000 0.1471058599075644


## Análisis de resultados

* En muchos casos la similaridad resultó ser 0, esto es porque se seleccionaron genes que tiene como $c_{MICA}$ a la raíz, el cual tiene información igual a cero.

* La similaridad de *Resnik* da el resultado esperado en las comparación entre funciones de un mismo nivel, pero **aumenta al descender en un nivel del grafo** ya que el ancestro pasa a ser un nodo más específico (en el caso de G2 vs G3). Lo que se esperaría de la similaridad al descender en el grafo es que disminuya puesto que las funciones están más alejadas.

* En *Lin* también se obtienen valores mayores de las comparaciones entre G2 y G3,  y **la comparación entre niveles es la esperada**, mayor en la de nivel 3.

* *JC* permite comparar bien entre funciones, pero en la comparación entre niveles ocurre que en el caso en donde el nodo raíz es el MICA la similaridad disminuye, lo cual por un lado tiene sentido porque agrega información sobre la distancia de las funciones, pero no define de forma única que las funciones no son similares. En la comparación de nivel 3 entre G2 y G3 se llega a un valor negativo, que si bien es mayor que la similaridad entre G1 y G2 **no refleja que tan parecido son los términos, y confunde con el signo del resultado**. Y para el caso de G2 y G3 de nivel 4 el valor aumenta cuando debería ser menor (funciones más específicas, menor similaridad).

* *Relevance* cumple con que las comparaciones con MICAs más cercanos de una similitud mayor,  y además el similaridad es menor a mayor profundidad (salvo en el caso en que MICA = root). El término de probabiladad del MICA produce una **similaridad mayor en términos con MICAs más específicos y menor en otros casos**.

# Similaridad sobre conjuntos de funciones 

## Medidas de similaridad

En este análisis se van a utilizar las medidas anteriores de Resnik, Lin y Schlicker, comparando entre pares de funciones y calculando promedio, máximo y mínimo, también se utiliza la medida del coseno definida a continuación.

**Coseno:**

$$ CoSim_{A,B} = \frac{A\cdot B}{|A|^2 + |B|^2 - A \cdot B}$$

In [0]:
import numpy as np

# genes
G1_go_id = [G1F1_go_id,G1F2_go_id]
G2_go_id = [G2F1_go_id,G2F2_go_id]
G3_go_id = [G3F1_go_id,G3F2_go_id]

G1vsG2_res = []
G1vsG2_lin = []
G1vsG2_rel = []
G1vsG2_cos = []

G3vsG2_res = []
G3vsG2_lin = []
G3vsG2_rel = []
G3vsG2_cos = []

for func1 in G1_go_id:
  for func2 in G2_go_id:
    G1vsG2_res.append(resnik_sim(func1, func2, godag, termcounts))
    G1vsG2_lin.append(lin_sim(func1, func2, godag, termcounts))
    G1vsG2_rel.append(rel_sim(func1, func2, godag, termcounts))
    
    
for func1 in G3_go_id:
  for func2 in G2_go_id:
    G3vsG2_res.append(resnik_sim(func1, func2, godag, termcounts))
    G3vsG2_lin.append(lin_sim(func1, func2, godag, termcounts))
    G3vsG2_rel.append(rel_sim(func1, func2, godag, termcounts))
    
G1G2_res_avg = sum(G1vsG2_res)/4
G3G2_res_avg = sum(G3vsG2_res)/4
G1G2_lin_avg = sum(G1vsG2_lin)/4
G3G2_lin_avg = sum(G3vsG2_lin)/4
G1G2_rel_avg = sum(G1vsG2_rel)/4
G3G2_rel_avg = sum(G3vsG2_rel)/4

G1G2_res_max = max(G1vsG2_res)
G3G2_res_max = max(G3vsG2_res)
G1G2_lin_max = max(G1vsG2_lin)
G3G2_lin_max = max(G3vsG2_lin)
G1G2_rel_max = max(G1vsG2_rel)
G3G2_rel_max = max(G3vsG2_rel)

G1G2_res_min = min(G1vsG2_res)
G3G2_res_min = min(G3vsG2_res)
G1G2_lin_min = min(G1vsG2_lin)
G3G2_lin_min = min(G3vsG2_lin)
G1G2_rel_min = min(G1vsG2_rel)
G3G2_rel_min = min(G3vsG2_rel)

from goatools.semantic import TermCounts, get_info_content
termcounts = TermCounts(godag, associations)
ic_G1F1 = get_info_content(G1F1_go_id, termcounts)
ic_G1F2 = get_info_content(G1F2_go_id, termcounts)
ic_G2F1 = get_info_content(G2F1_go_id, termcounts)
ic_G2F2 = get_info_content(G2F2_go_id, termcounts)
ic_G3F1 = get_info_content(G3F1_go_id, termcounts)
ic_G3F2 = get_info_content(G3F2_go_id, termcounts)


# se recorre del grafo en amplitud (ver figura),
# este recorrido determina la forma de los vectores
# se pesan los nodos con su contenido de información correspondiente
G1v = [0, 
       0, 0, 0, 
       0, 0, 
       0, 0, 0, 
       ic_G1F1, 0, 0,
       ic_G1F2, 0, 0,
       0, 0,
       0]

G2v = [0, 
       0, 0, 0, 
       0, 0, 
       0, ic_G2F1, 0, 
       0, ic_G2F2, 0,
       0, 0, 0,
       0, 0,
       0]

G3v = [0, 
       0, 0, 0, 
       0, 0, 
       0, 0, ic_G3F1, 
       0, 0, ic_G3F2,
       0, 0, 0,
       0, 0,
       0]

punto1 = np.array(G1v) @ np.array(G2v)
punto2 = np.array(G3v) @ np.array(G2v)

G1_snorm = 0
for x in G1v:
  G1_snorm += x**2
  
G2_snorm = 0
for x in G2v:
  G2_snorm += x**2
  
G3_snorm = 0
for x in G3v:
  G3_snorm += x**2
  
G1G2_cos = punto1 / (G1_snorm + G2_snorm - punto1)
G3G2_cos = punto2 / (G3_snorm + G2_snorm - punto2)

# informar resultados
print("Comparaciones de medidas por grupos, comparando contra G2")
print("   Measure           vs G1               vs G3       ")
print("------------- ------------------- -------------------")
print("resnik    min {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_res_min, sim2=G3G2_res_min))
print("lin       min {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_lin_min, sim2=G3G2_lin_min))
print("relevance min {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_rel_min, sim2=G3G2_rel_min))
print("")
print("resnik    avg {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_res_avg, sim2=G3G2_res_avg))
print("lin       avg {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_lin_avg, sim2=G3G2_lin_avg))
print("relevance avg {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_rel_avg, sim2=G3G2_rel_avg))
print("")
print("resnik    max {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_res_max, sim2=G3G2_res_max))
print("lin       max {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_lin_max, sim2=G3G2_lin_max))
print("relevance max {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_rel_max, sim2=G3G2_rel_max))
print("")
print("cosine        {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_cos, sim2=G3G2_cos))

Comparaciones de medidas por grupos, comparando contra G2
   Measure           vs G1               vs G3       
------------- ------------------- -------------------
resnik    min -0.0000000000000000  0.5447189755507806
lin       min -0.0000000000000000  0.3028250881078103
relevance min -0.0000000000000000  0.1271850967824201

resnik    avg 0.0000000000000000  0.5773162218079968
lin       avg 0.0000000000000000  0.4546588967294742
relevance avg 0.0000000000000000  0.1994070640116507

resnik    max -0.0000000000000000  0.6099134680652128
lin       max -0.0000000000000000  0.6014209012116800
relevance max -0.0000000000000000  0.2746100535072913

cosine        0.0000000000000000  0.0000000000000000


## Análisis de resultados

* En el caso de los genes que tienen como menor ancestro común al nodo raíz, su similaridad es cero.
* En el caso de la medida de coseno como los genes no tienen términos en común las similaridades se anulan.
* Para las medidas que usan max, min o avg, todos los casos son representativos, ya que los genes que se comparan son cercanos y no contienen términos en común.